In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,Maruti Swift Dzire AMT VDI,2018,50000,Diesel,Individual,Automatic,First Owner
2290,Honda City 1.5 V AT,2008,70000,Petrol,Individual,Automatic,First Owner
874,Maruti Alto 800 LXI,2017,50000,Petrol,Individual,Manual,First Owner
1907,Datsun GO T Petrol,2015,92198,Petrol,Dealer,Manual,Second Owner
3244,Hyundai EON Era Plus,2013,3240,Petrol,Individual,Manual,Second Owner


__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [5]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [6]:
import itertools
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric, target_name='selling_price'):
        ### Your code is here
        self.categorical = categorical
        self.numeric = numeric
        self.target_name = target_name
    
    def fit(self, X, y):
        ### Your code is here
        X_fit = X.copy()
        y_fit = y.copy()

        X_with_target = pd.concat((X_fit, y_fit), axis=1)

        self.dict_of_means = {col : (X_with_target.groupby(col)[self.target_name].mean()) for col in self.categorical}

        return self
        
    def transform(self, df):
        ### Your code is here
        X_ = df.copy()
        all_columns = self.categorical + self.numeric
        x_columns = X_.columns

        for col in self.categorical:
            X_[col] = X_[col].map(self.dict_of_means[col] + (np.random.rand() * 0.0006))
            mean_value = self.dict_of_means[col].values.mean() + (np.random.rand() * 0.0006)
            X_[col] = X_[col].fillna(0)

        missing_columns = [x for x in all_columns if x not in x_columns]

        extra_columns = [x for x in x_columns if x not in all_columns]

        ### Новые категории необходимо убрать
        X_ = X_.drop(extra_columns, axis=1)

        ### Отсутствующие категории (бинарные колонки)
        ### необходимо добавить: заполним их просто нулями

        if len(missing_columns) != 0:

            zeros = np.zeros((X_.shape[0], len(missing_columns)))
            zeros = pd.DataFrame(zeros,
                                 columns=missing_columns,
                                 index=X_.index)

            X_ = pd.concat((X_, zeros), axis=1)

        return X_

In [7]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,13.469369,13.439563,50000,13.089211,12.618208,13.771489,12.973187
2290,12.115704,11.910155,70000,12.453437,12.618208,13.771489,12.973187
874,12.302012,13.336728,50000,12.453437,12.618208,12.638427,12.973187
1907,12.484346,13.057348,92198,12.453437,13.147431,12.638427,12.460440
3244,12.392201,12.860755,3240,12.453437,12.618208,12.638427,12.460440
...,...,...,...,...,...,...,...
2895,11.862399,11.910155,40000,12.453437,12.618208,12.638427,12.973187
2763,13.145249,13.552865,15000,12.453437,12.618208,12.638427,12.973187
905,13.301476,13.057348,11918,12.453437,13.640955,12.638427,12.973187
3980,13.913664,13.057348,90000,13.089211,12.618208,12.638427,12.460440


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [8]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [9]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here
for max_depth in max_depth_list:
    pipe = Pipeline(
        [("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
         ("decision_tree", DecisionTreeRegressor(max_depth=max_depth))
         ])

    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f"Max depth {max_depth}, error {round(mse(y_true=y_test, y_pred=preds), 3)}")

print("======================================================")
for min_samples_split in min_samples_split_list:
    pipe = Pipeline(
        [("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
         ("decision_tree", DecisionTreeRegressor(min_samples_split=min_samples_split))
         ])

    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f"Min samples split {min_samples_split}, error {round(mse(y_true=y_test, y_pred=preds), 3)}")

print("======================================================")

for min_impurity_decrease in min_impurity_decrease_list:
    pipe = Pipeline(
        [("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
         ("decision_tree", DecisionTreeRegressor(min_impurity_decrease=min_impurity_decrease))
         ])

    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f"Min impurity decrease {min_impurity_decrease}, error {round(mse(y_true=y_test, y_pred=preds), 3)}")

print("======================================================")

for max_leaf_nodes in max_leaf_nodes_list:
    pipe = Pipeline(
        [("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
         ("decision_tree", DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes))
         ])

    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f"Max leaf nodes {max_leaf_nodes}, error {round(mse(y_true=y_test, y_pred=preds), 3)}")


Max depth 3, error 0.797
Max depth 5, error 1.442
Max depth 8, error 1.989
Max depth 12, error 1.982
Min samples split 10, error 1.431
Min samples split 50, error 1.439
Min samples split 100, error 0.952
Min samples split 500, error 0.808
Min impurity decrease 0, error 1.984
Min impurity decrease 0.1, error 0.52
Min impurity decrease 0.15, error 0.52
Min impurity decrease 0.2, error 0.52
Max leaf nodes 100, error 1.988
Max leaf nodes 200, error 1.984
Max leaf nodes 500, error 1.979


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here

pipe = Pipeline(
        [("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
         ("decision_tree", DecisionTreeRegressor())
         ])

search = GridSearchCV(pipe, param_grid, scoring='neg_mean_squared_error',)

search.fit(X_train, y_train)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

print(f"Качество лучшей модели на финальном тесте: {search.score(X_test, y_test)}")

Best parameter (CV score=-0.57274):
{'decision_tree__max_depth': 5, 'decision_tree__max_leaf_nodes': 500, 'decision_tree__min_impurity_decrease': 0.2, 'decision_tree__min_samples_split': 100}
Качество лучшей модели на финальном тесте: -0.5204871413036706
